In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

import json
import glob
from concurrent.futures import ThreadPoolExecutor
import os

import inflection
import pandas as pd
import pydicom

In [3]:
DIR_BASE = "../scratch/prototyping-sample/"

PATHS = glob.glob(os.path.join(DIR_BASE, "*"))

In [7]:
def jsonify_sequence(sequence):
    if len(sequence) == 0:
        return []
    return [datum.to_json_dict() for datum in sequence]


def parse_image_metadata(path):
    
    dcm = pydicom.dcmread(path)

    fields_camel = [field for field in dir(dcm) if field.lower() != field and not field.startswith("_")]
    fields_snake = [inflection.underscore(field) for field in fields_camel]

    data = {}
    for camel, snake in zip(fields_camel, fields_snake):
        elem = getattr(dcm, camel)
        if isinstance(elem, pydicom.sequence.Sequence):
            data[snake] = json.dumps(jsonify_sequence(elem))
        elif isinstance(elem, (str, int)):
            data[snake] = elem
        elif isinstance(elem, pydicom.valuerep.PersonName):
            data[snake] = str(elem)
        elif isinstance(elem, pydicom.multival.MultiValue):
            data[snake] = json.dumps(list(elem))
        elif camel == "PixelData":
            pass
        else:
            raise RuntimeError
            
    return data

In [11]:
%%time

records = [parse_image_metadata(path) for path in PATHS]
metadata = pd.DataFrame.from_records(records)
metadata.to_parquet("test.parquet")

CPU times: user 1.62 s, sys: 175 ms, total: 1.79 s
Wall time: 1.8 s
